In [21]:
import warnings
warnings.filterwarnings("ignore")

# Question 1

In [22]:
import pandas as pd

df = pd.read_excel("ds-challenge.xlsx")
df

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1.0,53.0,746.0,224.0,2.0,cash,2017-03-13 12:36:56.190
1,2.0,92.0,925.0,90.0,1.0,cash,2017-03-03 17:38:51.999
2,3.0,44.0,861.0,144.0,1.0,cash,2017-03-14 04:23:55.595
3,4.0,18.0,935.0,156.0,1.0,credit_card,2017-03-26 12:43:36.649
4,5.0,18.0,883.0,156.0,1.0,credit_card,2017-03-01 04:35:10.773
...,...,...,...,...,...,...,...
4995,4996.0,73.0,993.0,330.0,2.0,debit,2017-03-30 13:47:16.597
4996,4997.0,48.0,789.0,234.0,2.0,cash,2017-03-16 20:36:16.389
4997,4998.0,56.0,867.0,351.0,3.0,cash,2017-03-19 05:42:42.228
4998,4999.0,60.0,825.0,354.0,2.0,credit_card,2017-03-16 14:51:18.188


### Exploratory Data Analysis

In [23]:
# Exactly 100 shops
df['shop_id'].nunique()

100

In [24]:
# Check created date time (one month full)
df['created_at'] = pd.to_datetime(df['created_at'])
date_range = str(df['created_at'].dt.date.min()) + ' to ' + str(df['created_at'].dt.date.max())
date_range

'2017-03-01 to 2017-03-30'

In [25]:
# Check if each shop only sells one kind of shoe model
df.sort_values(by="shop_id").head(10)

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
4515,4516.0,1.0,797.0,316.0,2.0,cash,2017-03-17 16:43:51.449
4657,4658.0,1.0,886.0,316.0,2.0,debit,2017-03-02 19:20:23.838
1190,1191.0,1.0,734.0,316.0,2.0,debit,2017-03-03 03:40:14.564
347,348.0,1.0,907.0,158.0,1.0,credit_card,2017-03-03 06:59:34.915
2587,2588.0,1.0,756.0,316.0,2.0,debit,2017-03-30 12:27:18.832
41,42.0,1.0,847.0,316.0,2.0,debit,2017-03-20 14:58:02.406
2759,2760.0,1.0,877.0,474.0,3.0,cash,2017-03-09 17:31:50.985
948,949.0,1.0,930.0,474.0,3.0,cash,2017-03-29 22:46:56.056
388,389.0,1.0,713.0,316.0,2.0,cash,2017-03-12 22:22:49.457
3316,3317.0,1.0,918.0,316.0,2.0,debit,2017-03-15 07:44:37.085


We see that at index 4515, the total items is 2 with price as 316. On the other hand, at index 347, the total items is 1 with price 158 which is half of 316. In this case, we will assume that a shop only sell one type of sneaker at a specific price. We will validate our assumption by:

In [26]:
# We will divide all order amount by total items and see if it is the same
df1 = df.copy()
df1['order_amount'] = df1['order_amount'] / df1['total_items']
df1['total_items'] = 1.0
df1.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1.0,53.0,746.0,112.0,1.0,cash,2017-03-13 12:36:56.190
1,2.0,92.0,925.0,90.0,1.0,cash,2017-03-03 17:38:51.999
2,3.0,44.0,861.0,144.0,1.0,cash,2017-03-14 04:23:55.595
3,4.0,18.0,935.0,156.0,1.0,credit_card,2017-03-26 12:43:36.649
4,5.0,18.0,883.0,156.0,1.0,credit_card,2017-03-01 04:35:10.773


In [27]:
# For each shop
for shop_num in df1['shop_id'].unique():
    tmp = df1[df1['shop_id'] == shop_num]

    # If order_amount is not 1
    if tmp['order_amount'].nunique() != 1:
        print("Assumption is false.")
        print(shop_num)

In [28]:
price_table = df1[['shop_id', 'order_amount']]
price_table.rename(columns={'order_amount': 'price_per_item'}, inplace=True)
price_table.drop_duplicates(inplace=True)

Since there is no output, our assumption is then applicable. 

In [29]:
# We also check how many users made transaction(s)
df['user_id'].nunique()

301

### Problem in the problem statement -- Recursive, oops.
The naive way stated in the problem do AOV (average order value) calculation to the whole dataset, in which:

In [30]:
# We will review the dataset again
df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1.0,53.0,746.0,224.0,2.0,cash,2017-03-13 12:36:56.190
1,2.0,92.0,925.0,90.0,1.0,cash,2017-03-03 17:38:51.999
2,3.0,44.0,861.0,144.0,1.0,cash,2017-03-14 04:23:55.595
3,4.0,18.0,935.0,156.0,1.0,credit_card,2017-03-26 12:43:36.649
4,5.0,18.0,883.0,156.0,1.0,credit_card,2017-03-01 04:35:10.773


In [31]:
df.shape

(5000, 7)

In [32]:
# The naive way to calculate AOV
df['order_amount'].sum() / df.shape[0]

3145.128

### Question 1a

This is not the right way to use AOV. We can not gain any insight because we mixed the shops altogether. Hence, to make it more useful to each owner of the shops, we can transform the data to:

In [34]:
# First we see the data summary
df.describe()

,order_id,shop_id,user_id,order_amount,total_items
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000
mean,2500.500000,50.078800,849.092400,3145.128000,8.78720
std,1443.520003,29.006118,87.798982,41282.539349,116.32032
min,1.000000,1.000000,607.000000,90.000000,1.00000
25%,1250.750000,24.000000,775.000000,163.000000,1.00000
50%,2500.500000,50.000000,849.000000,284.000000,2.00000
75%,3750.250000,75.000000,925.000000,390.000000,3.00000
max,5000.000000,100.000000,999.000000,704000.000000,2000.00000


Seems like there are outliers in the order amount and total items. Let us remove them first.

In [36]:
# Remove outlier
summ = df.describe().T
summ['cut_off'] = summ.apply(lambda x: 1.5 * (x['75%'] - x['25%']), axis=1)
summ['higher_bound'] = summ.apply(lambda x: x['75%'] + x['cut_off'], axis=1)
summ['lower_bound'] = summ.apply(lambda x: x['25%'] - x['cut_off'], axis=1)
summ

,count,mean,std,min,25%,50%,75%,max,cut_off,higher_bound,lower_bound
order_id,4859.0,2497.395966,1443.356555,1.0,1244.5,2498.0,3749.5,5000.0,3757.5,7507.0,-2513.0
shop_id,4859.0,49.852645,29.049171,1.0,24.0,50.0,74.0,100.0,75.0,149.0,-51.0
user_id,4859.0,849.905742,86.887496,700.0,776.0,850.0,925.0,999.0,223.5,1148.5,552.5
order_amount,4859.0,293.715374,144.453395,90.0,162.0,280.0,380.0,730.0,327.0,707.0,-165.0
total_items,4859.0,1.950196,0.919791,1.0,1.0,2.0,3.0,5.0,3.0,6.0,-2.0


In [37]:
# Since there are no minus total item or order amount, we dont use lower bound
ti_hbound = summ.loc['total_items']['higher_bound']
od_hbound = summ.loc['order_amount']['higher_bound']
df = df[(df['total_items'] <= ti_hbound) & (df['order_amount'] <= od_hbound)]
df

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1.0,53.0,746.0,224.0,2.0,cash,2017-03-13 12:36:56.190
1,2.0,92.0,925.0,90.0,1.0,cash,2017-03-03 17:38:51.999
2,3.0,44.0,861.0,144.0,1.0,cash,2017-03-14 04:23:55.595
3,4.0,18.0,935.0,156.0,1.0,credit_card,2017-03-26 12:43:36.649
4,5.0,18.0,883.0,156.0,1.0,credit_card,2017-03-01 04:35:10.773
...,...,...,...,...,...,...,...
4995,4996.0,73.0,993.0,330.0,2.0,debit,2017-03-30 13:47:16.597
4996,4997.0,48.0,789.0,234.0,2.0,cash,2017-03-16 20:36:16.389
4997,4998.0,56.0,867.0,351.0,3.0,cash,2017-03-19 05:42:42.228
4998,4999.0,60.0,825.0,354.0,2.0,credit_card,2017-03-16 14:51:18.188


In [ ]:
df2 = (df[['shop_id', 'order_amount']]
        .groupby(by="shop_id")
        .sum())
df2['num_of_purchase'] = (df[['shop_id', 'created_at']]
                            .groupby(by="shop_id")
                            .count())
df2['shop_aov'] = df2['order_amount'] / df2['num_of_purchase'] 
df2.sort_values(by='shop_aov', ascending=False) # Each order value in a month

,order_amount,num_of_purchase,shop_aov
shop_id,,,
42.0,14432.0,28,515.428571
90.0,18868.0,48,393.083333
81.0,22656.0,59,384.000000
50.0,15247.0,41,371.878049
33.0,14186.0,39,363.743590
...,...,...,...
53.0,14560.0,68,214.117647
100.0,8547.0,40,213.675000
32.0,7979.0,42,189.976190


### Question 1b

There are some metrics which I consider to report the dataset, such as revenue per visitor (RPV) and average revenue per user (ARPU). However, RPV can not be calculated from the dataset since there are no information about number of visitors (including those who did not buy). On the other side, ARPU is not suitable as the performance indicator since sneaker shops are not subscribe-based (people rarely buy sneakers monthly).
<br>

Hence, I think average order value (AOV) still give better insights according to the dataset. 

### Question 1c

Each shop AOV has already calculated in Q.1a.

# Question 2

Question 2a: 54 orders 
<br> Query:

In [ ]:
"""
WITH SpeedyId as (
    SELECT ShipperId from Shippers
    WHERE ShipperName = "Speedy Express"
)

SELECT COUNT(od.OrderId) as "Number of orders shipped by Speedy Express" 
FROM Orders as od
JOIN SpeedyId as s ON s.ShipperId = od.ShipperId;
"""
print("", end="")

Question 2b: Peacock
<br> Query

In [ ]:
"""
WITH MostOrder as (
    SELECT EmployeeID
    FROM Orders
    GROUP BY EmployeeID
    ORDER BY COUNT(OrderId) DESC
    LIMIT 1
)

SELECT em.LastName as "Last name of employee with most order"
FROM Employees as em 
JOIN MostOrder as mo ON mo.EmployeeID = em.EmployeeID;
"""
print("", end="")

Question 2c: Gorgonzola Telino
<br> Query

In [ ]:
"""
WITH MostProductOrdered as (
	SELECT MAX(OrderCount), ProductID FROM (
        SELECT det.ProductID as ProductID, COUNT(det.OrderID) as OrderCount
        FROM Customers as cust
        JOIN Orders as ord ON cust.CustomerID = ord.CustomerID
        JOIN OrderDetails as det ON ord.OrderID = det.OrderID
        WHERE cust.Country = "Germany"
        GROUP BY det.ProductID
    )
)

SELECT prod.ProductName as "Most product sold in Germany"
FROM MostProductOrdered as mpo
JOIN Products as prod ON mpo.ProductID = prod.ProductID;
"""
print("", end="")